In [7]:
import numpy as np
from doubleml.datasets import fetch_bonus
from doubleml import DoubleMLData
import statsmodels.api as sm
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LassoCV
from doubleml import DoubleMLPLR
from doubleml import DoubleMLPLR
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
from sklearn.linear_model import LinearRegression, LogisticRegression

In [14]:
np.random.seed(1)
df = fetch_bonus('DataFrame')
df.head(5)

import pandas as pd
import numpy as np
df = pd.read_csv('/Users/pranjal/Desktop/Causal-Inference/data/nyvoucher.csv')
outcome = 'post_ach'
treatment = 'voucher'
rest = list(df.drop([outcome, treatment, 'pre_ach'], axis = 1).columns)
df = df[[outcome] + [treatment] + rest]
df = df.dropna()
y = df[outcome]
d = df[treatment]
x = df[rest].astype('float')

l = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
g = clone(l)
m = RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', max_depth= 5)

def score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

print(y.shape, x.shape, d.shape)

(521,) (521, 1) (521,)


In [15]:
df

,post_ach,voucher,s_id
0,83.0,0,42
1,4.0,0,194
2,3.5,1,218
3,26.5,1,261
4,2.0,1,304
...,...,...,...
516,24.0,1,21796
517,12.5,0,21808
518,54.5,1,21814
519,6.0,1,21816


In [2]:
OLS = sm.OLS(y,sm.add_constant(np.c_[d,x]), prepend=False).fit()

NameError: name 'sm' is not defined

In [17]:
data = DoubleMLData(df.sample(int(df.shape[0]*0.8)),
                  y_col='post_ach',
                  d_cols='voucher',
                  x_cols=rest)
SML = DoubleMLPLR(data, l,m,g, n_folds=1,apply_cross_fitting=False,score=score)
SML.fit();

In [19]:
data = DoubleMLData(df.sample(int(df.shape[0]*0.8)),
                  y_col='post_ach',
                  d_cols='voucher',
                  x_cols=rest)

OML = DoubleMLPLR(data, l,m,g, n_folds=1,apply_cross_fitting=False,score='IV-type')
OML.fit();

In [20]:
data = DoubleMLData(df.sample(int(df.shape[0]*1)),
                  y_col='post_ach',
                  d_cols='voucher',
                  x_cols=rest)
DML = DoubleMLPLR(data, l,m,g, n_folds=5,apply_cross_fitting=True,score='IV-type')
DML.fit();

In [21]:
table = PrettyTable()
table.field_names = ['Estimator', 'θ_hat', 'Std Error','t','p','2.5%','97.25%']
a = ['OLS']+ np.c_[OLS.params[1], OLS.bse[1], OLS.tvalues[1], OLS.pvalues[1], np.nan, np.nan].reshape(-1).tolist()
table.add_row(a)
a = ['Naive/Single ML (SML)']+ np.array(SML.summary).reshape(-1).tolist()
table.add_row(a)
a = ['Orthogonal ML (OML)']+ np.array(OML.summary).reshape(-1).tolist()
table.add_row(a)
a = ['Orthogonal+Crossfitting ML (DML)']+ np.array(DML.summary).reshape(-1).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+----------------------------------+-------+-----------+-------+-------+-------+--------+
|            Estimator             | θ_hat | Std Error |   t   |   p   |  2.5% | 97.25% |
+----------------------------------+-------+-----------+-------+-------+-------+--------+
|               OLS                | 4.934 |   1.688   | 2.923 | 0.004 |  nan  |  nan   |
|      Naive/Single ML (SML)       | 4.920 |   1.172   | 4.199 | 0.000 | 2.623 | 7.216  |
|       Orthogonal ML (OML)        | 7.575 |   1.738   | 4.359 | 0.000 | 4.169 | 10.982 |
| Orthogonal+Crossfitting ML (DML) | 5.461 |   1.747   | 3.127 | 0.002 | 2.038 | 8.884  |
+----------------------------------+-------+-----------+-------+-------+-------+--------+
